# Case Study - Regression

This data is a **regression problem**, trying to predict life expectancy.

The followings describe the features.

- **Country**
- **Year**
- **Status**: Developed or Developing status
- **Life expectancy**: Life Expectancy in age
- **Adult Mortality**: Adult Mortality Rates of both sexes (probability of dying between 15 and 60 years per 1000 population)
- **Infant deaths**: Number of Infant Deaths per 1000 population
- **Alcohol**: Alcohol, recorded per capita (15+) consumption (in litres of pure alcohol)
- **Percentage expenditure**: Expenditure on health as a percentage of Gross Domestic Product per capita(%)
- **Hepatitis B**: Hepatitis B (HepB) immunization coverage among 1-year-olds (%)
- **Measles**: - number of reported cases per 1000 population
- **BMI** Average Body Mass Index of entire population
- **under-five deaths**: Number of under-five deaths per 1000 population
- **Polio**: Polio (Pol3) immunization coverage among 1-year-olds (%)
- **Total expenditure**: General government expenditure on health as a percentage of total government expenditure (%)
- **Diphtheria**: Diphtheria tetanus toxoid and pertussis (DTP3) immunization coverage among 1-year-olds (%)
- **HIV/AIDS**: Deaths per 1000 live births HIV/AIDS (0-4 years)
- **GDP**: Gross Domestic Product per capita (in USD)
- **Population**: Population of the country
- **thinness 1-19 years**: Prevalence of thinness among children and adolescents for Age 10 to 19 (% )
- **thinness 5-9 years**: Prevalence of thinness among children for Age 5 to 9(%)
- **Income composition of resources**: Human Development Index in terms of income composition of resources (index ranging from 0 to 1)
- **Schooling**: Number of years of Schooling(years)

## Importing libraries

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import matplotlib
np.__version__, pd.__version__, sns.__version__, matplotlib.__version__

## 1. Load data

In [ ]:
df = pd.read_csv('data/car_price_dataset.csv')

In [ ]:
# print the first rows of data
df.head()

In [ ]:
# Check the shape of your data
df.shape

In [ ]:
# Statistical info Hint: look up .describe()
df.describe()

In [ ]:
# Check Dtypes of your input data
df.info()

In [ ]:
# Check the column names
df.columns

## 2. Exploratory Data Analysis

EDA is an essential step to inspect the data, so to better understand nature of the given data.

### Renaming

Now we would like to rename some of the following column names, so it's easy to write the code...

In [ ]:
df.columns

In [ ]:
# rename columns
df.rename(columns = {'name':'brand' 
                     }, inplace = True)

In [ ]:
df.columns

### 2.1 Univariate analyis

Single variable exploratory data anlaysis

#### Countplot

In [ ]:
# Let's see how many developing and developed countries there are
sns.countplot(data = df, x = 'seats')

#### Distribution plot

In [ ]:
sns.displot(data = df, x = 'seats')

### 2.2 Multivariate analysis

Multiple variable exploratory data analysis

#### Boxplot

In [ ]:
# Let's try bar plot on "Status"
sns.boxplot(x = df["transmission"], y = df["selling_price"])
plt.ylabel("selling_price")
plt.xlabel("transmission")

#### Scatterplot

In [ ]:
sns.scatterplot(x = df['selling_price'], y = df['year'], hue=df['transmission'])

#### Correlation Matrix

Let's use correlation matrix to find strong factors predicting the life expectancy.  It's also for checking whether certain features are too correlated.

In [ ]:
# Let's check out heatmap
plt.figure(figsize = (15,8))
sns.heatmap(df.corr(), annot=True, cmap="coolwarm")  #don't forget these are not all variables! categorical is not here...

#### Tips: Label encoding

Now we would like to change "Developing" and "Developed" to "0" and "1", since machine learning algorithms do not understand text.   Also, correlation matrix and other similar computational tools require label encoding.

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_mapping = {
    'First Owner': 1,
    'Second Owner': 2,
    'Third Owner': 3,
    'Fourth & Above Owner': 4,
    'Test Drive Car': 5
}

categorical_data = df['owner']

df['owner'] = [label_mapping[label] for label in categorical_data]

In [ ]:
df = df[~df['fuel'].isin(['CNG', 'LPG'])]

In [ ]:
# Extract numeric mileage values by splitting and converting to float
df['mileage'] = df['mileage'].str.split().str[0].astype(float)

In [ ]:
# Remove "CC" and convert to float
df['engine'] = df['engine'].str.replace(' CC', '').astype(float)

In [ ]:
# Remove " bhp" and convert to float, handling N/A values
df['max_power'] = df['max_power'].str.replace(' bhp', '').astype(float)

In [ ]:
# Extract the first word and update the column
df['brand'] = df['brand'].apply(lambda x: x.split()[0])

In [ ]:
label_mapping = {
    'Diesel': 1,
    'Petrol': 2
}

categorical_data1 = df['fuel']

df['lable_fuel'] = [label_mapping[label] for label in categorical_data1]

In [ ]:
# Drop the 'torque' feature
df = df.drop(columns=['torque'])

In [ ]:
# Remove rows with 'Test Drive Cars' in the 'make' column
df = df[df['owner'] != 5]

In [ ]:
import numpy as np
df['log_selling_price'] = np.log(df['selling_price'])

In [ ]:
df.head(45)

In [ ]:
# Let's check out heatmap
plt.figure(figsize = (15,8))
sns.heatmap(df.corr(), annot=True, cmap="coolwarm")  #don't forget these are not all variables! categorical is not here...

#### Predictive Power Score

This is another way to check the predictive power of some feature.  Unlike correlation, `pps` actually obtained from actual prediction.  For more details:
    
- The score is calculated using only 1 feature trying to predict the target column. This means there are no interaction effects between the scores of various features. Note that this is in contrast to feature importance
- The score is calculated on the test sets of a 4-fold crossvalidation (number is adjustable via `ppscore.CV_ITERATIONS`)
- All rows which have a missing value in the feature or the target column are dropped
- In case that the dataset has more than 5,000 rows the score is only calculated on a random subset of 5,000 rows with a fixed random seed (`ppscore.RANDOM_SEED`). You can adjust the number of rows or skip this sampling via the API. However, in most scenarios the results will be very similar.
- There is no grid search for optimal model parameters

We can install by doing <code>pip install ppscore</code>

In [ ]:
import ppscore as pps

# before using pps, let's drop country and year
dfcopy = df.copy()
dfcopy.drop(['year'], axis='columns', inplace=True)

#this needs some minor preprocessing because seaborn.heatmap unfortunately does not accept tidy data
matrix_df = pps.matrix(dfcopy)[['x', 'y', 'ppscore']].pivot(columns='x', index='y', values='ppscore')

#plot
plt.figure(figsize = (15,8))
sns.heatmap(matrix_df, vmin=0, vmax=1, cmap="Blues", linewidths=0.5, annot=True)

## 3. Feature Engineering

We gonna skip for this tutorial.  But we can certainly try to combine some columsn to create new features.

## 4. Feature selection

In [ ]:
#x is our strong features
X = df[['max_power','engine','mileage','seats', 'km_driven', 'owner', 'lable_fuel']]

#y is simply the life expectancy col
y = df["log_selling_price"]

### Train test split

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.4, random_state = 42) #It can be 0.2-0.4 for test_size

## 5. Preprocessing

### Null values

In [ ]:
#check for null values
X_train[['max_power']].isna().sum()

In [ ]:
X_test[['max_power']].isna().sum()

In [ ]:
#check for null values
X_train[['engine']].isna().sum()

In [ ]:
X_test[['engine']].isna().sum()

In [ ]:
#check for null values
X_train[['mileage']].isna().sum()

In [ ]:
X_test[['mileage']].isna().sum()

In [ ]:
#check for null values
X_train[['seats']].isna().sum()

In [ ]:
X_test[['seats']].isna().sum()

In [ ]:
#check for null values
X_train[['km_driven']].isna().sum()

In [ ]:
X_test[['km_driven']].isna().sum()

In [ ]:
#check for null values
X_train[['owner']].isna().sum()

In [ ]:
X_test[['owner']].isna().sum()

In [ ]:
#check for null values
X_train[['lable_fuel']].isna().sum()

In [ ]:
X_test[['lable_fuel']].isna().sum()

In [ ]:
y_train.isna().sum()

In [ ]:
y_test.isna().sum()

In [ ]:
sns.displot(data=df, x='max_power')

In [ ]:
df['max_power'].mean(),df['max_power'].median()

In [ ]:
sns.displot(data=df, x='engine')

In [ ]:
df['engine'].mean(),df['engine'].median()

In [ ]:
sns.displot(data=df, x='mileage')

In [ ]:
df['mileage'].mean(),df['mileage'].median()

In [ ]:
sns.displot(data=df, x='seats')

In [ ]:
df['seats'].mean(),df['seats'].median()

In [ ]:
sns.displot(data=df, x='km_driven')

In [ ]:
df['km_driven'].mean(),df['km_driven'].median()

In [ ]:
sns.displot(data=df, x='owner')

In [ ]:
df['owner'].mean(),df['owner'].median()

In [ ]:
sns.displot(data=df, x='lable_fuel')

In [ ]:
df['lable_fuel'].mean(),df['lable_fuel'].median()

In [ ]:
sns.displot(y_train)

In [ ]:
df['log_selling_price'].mean(),df['log_selling_price'].median()

Mean: Use the mean to fill missing values if the data is approximately normally distributed and does not have significant outliers. The mean is sensitive to extreme values, so if your data has outliers, using the mean might result in skewed imputations.

Median: Use the median to fill missing values if your data has outliers or is skewed. The median is a robust measure of central tendency and is less affected by extreme values compared to the mean.

In [ ]:
#let's fill the training set first!

X_train['max_power'].fillna(X_train['max_power'].median(), inplace=True)
X_train['engine'].fillna(X_train['engine'].median(), inplace=True)
X_train['mileage'].fillna(X_train['mileage'].median(), inplace=True)
X_train['seats'].fillna(X_train['seats'].median(), inplace=True)
X_train['km_driven'].fillna(X_train['km_driven'].median(), inplace=True)
X_train['owner'].fillna(X_train['owner'].mean(), inplace=True)
X_train['lable_fuel'].fillna(X_train['lable_fuel'].median(), inplace=True)

In [ ]:
#let's fill the testing set with the training distribution first!

X_test['max_power'].fillna(X_train['max_power'].median(), inplace=True)
X_test['engine'].fillna(X_train['engine'].median(), inplace=True)
X_test['mileage'].fillna(X_train['mileage'].median(), inplace=True)
X_test['seats'].fillna(X_train['seats'].median(), inplace=True)
X_test['km_driven'].fillna(X_train['km_driven'].median(), inplace=True)
X_test['owner'].fillna(X_train['owner'].mean(), inplace=True)
X_test['lable_fuel'].fillna(X_train['lable_fuel'].median(), inplace=True)

In [ ]:
#same for y
y_train.fillna(y_train.median(), inplace=True)
y_test.fillna(y_train.median(), inplace=True)

In [ ]:
#check again
X_train[['max_power']].isna().sum()

In [ ]:
X_test[['max_power']].isna().sum()

In [ ]:
X_train[['engine']].isna().sum()

In [ ]:
X_test[['engine']].isna().sum()

In [ ]:
X_train[['mileage']].isna().sum()

In [ ]:
X_test[['mileage']].isna().sum()

In [ ]:
X_train[['seats']].isna().sum()

In [ ]:
X_test[['seats']].isna().sum()

In [ ]:
X_train[['km_driven']].isna().sum()

In [ ]:
X_test[['km_driven']].isna().sum()

In [ ]:
X_train[['owner']].isna().sum()

In [ ]:
X_test[['owner']].isna().sum()

In [ ]:
X_train[['lable_fuel']].isna().sum()

In [ ]:
X_test[['lable_fuel']].isna().sum()

In [ ]:
y_train.isna().sum(), y_test.isna().sum()

### Checking Outliers

In [ ]:
# Create a dictionary of columns.
col_dict = {'max_power':1,'engine':2,'mileage':3,'seats':4, 'km_driven':5, 'owner':6, 'lable_fuel':7}

# Detect outliers in each variable using box plots.
plt.figure(figsize=(20,30))

for variable,i in col_dict.items():
                     plt.subplot(5,4,i)
                     plt.boxplot(X_train[variable])
                     plt.title(variable)

plt.show()

In [ ]:
def outlier_count(col, data = X_train):
    
    # calculate your 25% quatile and 75% quatile
    q75, q25 = np.percentile(data[col], [75, 25])
    
    # calculate your inter quatile
    iqr = q75 - q25
    
    # min_val and max_val
    min_val = q25 - (iqr*1.5)
    max_val = q75 + (iqr*1.5)
    
    # count number of outliers, which are the data that are less than min_val or more than max_val calculated above
    outlier_count = len(np.where((data[col] > max_val) | (data[col] < min_val))[0])
    
    # calculate the percentage of the outliers
    outlier_percent = round(outlier_count/len(data[col])*100, 2)
    
    if(outlier_count > 0):
        print("\n"+15*'-' + col + 15*'-'+"\n")
        print('Number of outliers: {}'.format(outlier_count))
        print('Percent of data that is outlier: {}%'.format(outlier_percent))

In [ ]:
for col in X_train.columns:
    outlier_count(col)

### Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler


# feature scaling helps improve reach convergence faster
scaler = StandardScaler() # for standadization use StandardScaler() , for normalization use MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)

#x = (x - mean) / std
#why do we want to scale our data before data analysis / machine learning

#allows your machine learning model to catch the pattern/relationship faster
#faster convergence

#how many ways to scale
#standardardization <====current way
# (x - mean) / std
#--> when your data follows normal distribution

#normalization <---another way
# (x - x_min) / (x_max - x_min)
#---> when your data DOES NOT follow normal distribution (e.g., audio, signal, image) We will use nomalization when mean is a bad.

In [ ]:
# Let's check shapes of all X_train, X_test, y_train, y_test
print("Shape of X_train: ", X_train.shape)
print("Shape of X_test: ", X_test.shape)
print("Shape of y_train: ", y_train.shape)
print("Shape of y_test: ", y_test.shape)

## 6. Modeling

Let's define some algorithms and compare them using cross-validation.

[Scikit-Learn](http://scikit-learn.org) provides quick access to a huge pool of machine learning algorithms.

Before using sklearn, there is **one thing you need to know**, i.e., the **data shape that sklearn wants**.

To apply majority of the algorithms, sklearn requires two inputs, i.e., $\mathbf{X}$ and $\mathbf{y}$.

-  $\mathbf{X}$, or the **feature matrix** *typically* has the shape of ``[n_samples, n_features]``
-  $\mathbf{y}$, or the **target/label vector** *typically* has the shape of ``[n_samples, ]`` or ``[n_samples, n_targets]`` depending whether that algorithm supports multiple labels

Note 1:  if you $\mathbf{X}$ has only 1 feature, the shape must be ``[n_samples, 1]`` NOT ``[n_samples, ]``

Note 2:  sklearn supports both numpy and pandas, as long as the shape is right.  For example, if you use pandas, $\mathbf{X}$ would be a dataframe, and $\mathbf{y}$ could be a series or dataframe.

Tips:  it's always better to look at sklearn documentation before applying any algorithm.

### Much better: Cross validation + Grid search

In [ ]:
from sklearn.linear_model import LinearRegression  #we are using regression models
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

# Libraries for model evaluation

# models that we will be using, put them in a list
algorithms = [LinearRegression(), SVR(), KNeighborsRegressor(), DecisionTreeRegressor(random_state = 0), 
              RandomForestRegressor(n_estimators = 100, random_state = 0)]

# The names of the models
algorithm_names = ["Linear Regression", "SVR", "KNeighbors Regressor", "Decision-Tree Regressor", "Random-Forest Regressor"]

In [ ]:
from sklearn.linear_model import LinearRegression  #we are using regression models
from sklearn.metrics import mean_squared_error, r2_score

lr = LinearRegression()
lr.fit(X_train, y_train)
yhat = lr.predict(X_test)

print("MSE: ", mean_squared_error(y_test, yhat))
print("r2: ", r2_score(y_test, yhat))

Let's do some simple cross-validation here....

In [ ]:
#y_train.isna().sum()

In [ ]:
from sklearn.model_selection import KFold, cross_val_score

#lists for keeping mse
train_mse = []
test_mse = []

#defining splits
kfold = KFold(n_splits=5, shuffle=True)

for i, model in enumerate(algorithms):
    scores = cross_val_score(model, X_train, y_train, cv=kfold, scoring='neg_mean_squared_error') #Higher is better.
    print(f"{algorithm_names[i]} - Score: {scores}; Mean: {scores.mean()}")

Hmm...it seems random forest do very well....how about we grid search further to find the best version of the model.

### Grid Search

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {'bootstrap': [True], 'max_depth': [5, 10, None],
              'n_estimators': [5, 6, 7, 8, 9, 10, 11, 12, 13, 15]}

rf = RandomForestRegressor(random_state = 1)

grid = GridSearchCV(estimator = rf, 
                    param_grid = param_grid, 
                    cv = kfold, 
                    n_jobs = -1, 
                    return_train_score=True, 
                    refit=True,
                    scoring='neg_mean_squared_error')

# Fit your grid_search
grid.fit(X_train, y_train);  #fit means start looping all the possible parameters

In [ ]:
grid.best_params_

In [ ]:
# Find your grid_search's best score
best_mse = grid.best_score_

In [ ]:
best_mse  # ignore the minus because it's neg_mean_squared_error

## 7. Testing

Of course, once we do everything.  We can try to shoot with the final test set.  We should no longer do anything like improving the model.  It's illegal!  since X_test is the final final test set.

In [ ]:
yhat = grid.predict(X_test)
mean_squared_error(y_test,yhat)

In [ ]:
print(yhat)

In [ ]:
y_pred_original = np.exp(yhat)

In [ ]:
print(y_pred_original)

## 8. Analysis:  Feature Importance

Understanding why is **key** to every business, not how low MSE we got.  Extracting which feature is important for prediction can help us interpret the results.  There are several ways: algorithm, permutation, and shap.  Note that these techniques can be mostly applied to most algorithms. 

Most of the time, we just apply all, and check the consistency.

#### Algorithm way

Some ML algorithms provide feature importance score after you fit the model

In [ ]:
#stored in this variable
#note that grid here is random forest
rf = grid.best_estimator_

rf.feature_importances_

In [ ]:
#let's plot
plt.barh(X.columns, rf.feature_importances_)

In [ ]:
#hmm...let's sort first
sorted_idx = rf.feature_importances_.argsort()
plt.barh(X.columns[sorted_idx], rf.feature_importances_[sorted_idx])
plt.xlabel("Random-Forest Regressor Importance")

#### Permutation way

This method will randomly shuffle each feature and compute the change in the model’s performance. The features which impact the performance the most are the most important one.

*Note*: The permutation based importance is computationally expensive. The permutation based method can have problem with highly-correlated features, it can report them as unimportant.

In [ ]:
from sklearn.inspection import permutation_importance

perm_importance = permutation_importance(rf, X_test, y_test)

#let's plot
sorted_idx = perm_importance.importances_mean.argsort()
plt.barh(X.columns[sorted_idx], perm_importance.importances_mean[sorted_idx])
plt.xlabel("Random-Forest Regressor Importance")

#### Shap way

The SHAP interpretation can be used (it is model-agnostic) to compute the feature importances. It is using the Shapley values from game theory to estimate the how does each feature contribute to the prediction. It can be easily installed (<code>pip install shap</code>) 

In [ ]:
import shap

explainer = shap.TreeExplainer(rf)
shap_values = explainer.shap_values(X_test)

In [ ]:
#shap provides plot
shap.summary_plot(shap_values, X_test, plot_type="bar", feature_names = X.columns)

## 9. Inference

To provide inference service or deploy, it's best to save the model for latter use.

In [ ]:
import pickle

# save the model to disk
filename = 'model/price_car_prediction.model'
pickle.dump(grid, open(filename, 'wb'))

In [ ]:
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))

In [ ]:
#let's try to create one silly example
df[['max_power','engine','mileage','seats', 'km_driven', 'owner', 'lable_fuel','selling_price']].loc[1]

In [ ]:
sample = np.array([[103.52,1498.00,21.14,5.00,120000.00,2,1]])

In [ ]:
selling_price = loaded_model.predict(sample)
selling_price = np.exp(selling_price)
print(selling_price)